# Slash Command API Playground

Run slash queries directly against the running Cerebros backend from this notebook. Update the `API_BASE` below if your server uses a different host/port.


In [ ]:
import json
import os
from typing import Any, Dict

import requests
from websockets.sync.client import connect as ws_connect

API_BASE = os.environ.get("CEREBROS_API_BASE", "http://localhost:8000")
STRUCTURED_TYPES = {"slash_slack_summary", "slash_git_summary", "slash_cerebros_summary"}


def _post_preview(payload: Dict[str, Any]) -> Dict[str, Any]:
    response = requests.post(f"{API_BASE}/api/commands/preview", json=payload, timeout=30)
    response.raise_for_status()
    return response.json()


def _extract_structured(event: Dict[str, Any]) -> Dict[str, Any]:
    candidate = event.get("result") if isinstance(event.get("result"), dict) else event
    if candidate.get("type") in STRUCTURED_TYPES:
        return candidate
    return {}


def _ws_chat(payload: Dict[str, Any]) -> Dict[str, Any]:
    url = API_BASE.replace("http", "ws") + "/ws/chat"
    with ws_connect(url) as ws:
        ws.send(json.dumps({"message": payload["task"], "session_id": payload.get("session_id")}))
        for raw in ws:
            event = json.loads(raw)
            structured = _extract_structured(event)
            if structured:
                return structured
            if event.get("type") == "error":
                raise RuntimeError(event.get("message", "WebSocket error"))
    raise RuntimeError("WebSocket closed before a slash summary was received")


def run_slash_query(query: str, session_id: str = "notebook-session") -> Dict[str, Any]:
    payload = {"command": "slack", "task": query, "session_id": session_id}
    try:
        return _post_preview(payload)
    except requests.HTTPError as err:
        print("Preview endpoint unavailable, falling back to websocket (", err, ")")
    return _ws_chat(payload)


def display_result(result: Dict[str, Any]) -> None:
    print(f"Type: {result.get('type')}")
    print(f"Message: {result.get('message')}")
    sections = result.get("sections") or {}
    for section, items in sections.items():
        if not items:
            continue
        print(f"\n## {section.title()}")
        for item in items:
            print("-", json.dumps(item, indent=2))
    sources = result.get("sources") or []
    if sources:
        print("\nSources:")
        for src in sources:
            print("-", src.get("channel"), src.get("permalink"))
